# Publish PlanetScope Imagery to ArcGIS Image for ArcGIS Online

This script takes activated Planet orders and publishes them as Image Services with ArcGIS Online.  With Planet imagery published in ArcGIS Online, you are able to:

* Use the imagery in analytics workflows using raster functions or raster analytics
* Access full bit-depth imagery for custom stretching or band combinations performed on the fly
* Securely share imagery with your end-users since it is hosted inside of ArcGIS Online

This script specifically works with PlanetScope 8-band analytics surface reflectance assets, but could be modified to work with additional asset types.  For example, this could be extended to support Planet Basemaps or SkySat imagery.

### Prerequisites

* Access to ArcGIS Online with an [ArcGIS Image for ArcGIS Online license](https://www.esri.com/en-us/arcgis/products/arcgis-image/options/arcgis-online)
* A previously placed order for PlanetScope 8-band imagery, either through our [Order's API](https://developers.planet.com/docs/orders/), [ArcGIS Pro Integration](https://developers.planet.com/docs/integrations/arcgis/), or [Explorer](https://www.planet.com/explorer)
* Edit the config.py file in this notebooks folder which is used to store credentials for ArcGIS Online and Planet's platform.


In [91]:
import arcgis
from arcgis.gis import GIS
from arcgis.raster.analytics import copy_raster, create_image_collection, list_datastore_content
import os
import planet 
import asyncio
import glob
from zipfile import ZipFile
from pathlib import Path
import pandas as pd
import datetime

In [ ]:
# Connect to Planet API

pl_api_key = os.environ.get("PL_API_KEY")
plsdk_auth = planet.Auth.from_key(key=pl_api_key)
sess = planet.Session(plsdk_auth)
pl = planet.Planet(sess)

First, you need to provide an order ID to publish.  You could get the order ID from:

* Manually from [Planet Explorer](https://www.planet.com/explorer) or your [planet.com account orders page](https://www.planet.com/account/#/orders)
* Or programatically from Planet's Orders API

For example, a script could be used to search for orders from with the last 24 hours to be published to ArcGIS Online.

For this script, choose an order for the asset type <code>analytic_8b_sr_udm2</code> and which was not delivered to hosted data.

In [57]:
# Collect all orders
all_orders = []

# Get the generator from list_orders
orders_generator = pl.orders.list_orders()

# Iterate through the generator to get each order
for order in orders_generator:
    all_orders.append(order)

# Extract relevant fields into a list of dictionaries
orders_data = []
for order in all_orders:
    # Extract product details if available
    item_type = "N/A"
    product_bundle = "N/A"
    if order.get('products') and len(order['products']) > 0:
        item_type = order['products'][0].get('item_type', 'N/A')
        product_bundle = order['products'][0].get('product_bundle', 'N/A')
    
    # Create a simplified dictionary with just the fields we want
    order_info = {
        'ID': order.get('id', 'N/A'),
        'Created': order.get('created_on', 'N/A'),
        'Name': order.get('name', 'N/A'),
        'State': order.get('state', 'N/A'),
        'Item Type': item_type,
        'Product Bundle': product_bundle
    }
    orders_data.append(order_info)

# Create and display DataFrame
orders_df = pd.DataFrame(orders_data)
orders_df

,ID,Created,Name,State,Item Type,Product Bundle
0,6bb2fac9-ce05-4700-b5db-2504f28223dd,2025-06-10T18:10:30.564596Z,"scenes order, direct download",success,PSScene,analytic_udm2
1,dfdf25e0-96c1-4fef-8d38-d65a26673916,2025-06-10T18:07:24.63035Z,San Francisco May 31,success,PSScene,analytic_8b_sr_udm2
2,2d2a4501-c0f6-4649-bec5-3a6a95be803e,2025-06-10T17:51:56.180156Z,Pheonix Order,success,PSScene,analytic_8b_sr_udm2


In [63]:
# Select the first order ID to use in the next steps, or specify your own
my_order_id = orders_df.iloc[0].ID
# my_order_id = "your-planet-order-id"
my_order_name = pl.orders.get_order(my_order_id)['name']

print(f'Order to be published: {my_order_name} - id: {my_order_id}')

Order to be published: scenes order, direct download - id: 6bb2fac9-ce05-4700-b5db-2504f28223dd


In [89]:
# Create download directory if it doesn't exist
download_dir = Path("./planet_downloads")
download_dir.mkdir(exist_ok=True)

# Create order-specific directory
order_dir = download_dir / my_order_id
order_dir.mkdir(exist_ok=True)

# Array to store local file paths
local_tiff_files = []

# Create an async Planet API session
async with planet.Session() as ps:
    
    # Create a Planet API client using the modern method
    client = ps.client('orders')
    
    # Get the order details and name
    order_details = await client.get_order(order_id=my_order_id)
    order_name = order_details['name']
    print(f"Processing order: {order_name}")
    print(f"All files will be stored in: {order_dir}")

    # Check if the order has been successfully completed
    if order_details['state'] != 'success':
        print("Order isn't completed yet")
        raise Exception("Order not ready for download")

    # Check if the order contains zip archives or direct file links
    zip_archives = [r['name'] for r in order_details['_links']['results'] if r['name'].endswith(".zip")]
    
    if len(zip_archives) > 0:
        print("Order contains zip archives - downloading and extracting...")
        
        # Create subfolders for organization
        raw_dir = order_dir / "raw_downloads"
        extracted_dir = order_dir / "extracted"
        raw_dir.mkdir(exist_ok=True)
        extracted_dir.mkdir(exist_ok=True)
        
        # Download all assets including zip archives to the raw downloads folder within order directory
        download_results = await client.download_order(
            my_order_id, 
            directory=raw_dir,  # Download to raw folder within order directory
            overwrite=True,
            progress_bar=True
        )
        
        # Create a list of all zip files that were downloaded
        zip_files = [x for x in download_results if x.suffix == ".zip"]
        
        # Extract each zip file to the extracted folder
        for zip_file in zip_files:
            print(f"Extracting {zip_file.name}...")
            with ZipFile(zip_file) as z:
                z.extractall(extracted_dir)  # Extract to extracted folder
        
        # Find all relevant tiff files in the extracted directory
        local_tiff_files = [file for file in extracted_dir.glob("**/*.tif") if "udm" not in file.name.lower()]

    else:
        print("Order contains direct file links - downloading individual files...")
        
        # Download each tiff file individually to the order directory
        for result in order_details['_links']['results']:
            filename = result['name']
            location = result['location']
            
            print(f"Downloading {filename}...")
            downloaded_file = await client.download_asset(
                location=location,
                filename=filename,
                directory=download_dir,  # Download directly to order directory
                overwrite=True,
                progress_bar=True
            )
            
        # Find all relevant tiff files in the extracted directory
        local_tiff_files = [file for file in order_dir.glob("**/*.tif") if "udm" not in file.name.lower()]
        
print(f"\nDownload complete! Found {len(local_tiff_files)} SR tiff files:")
for file_path in local_tiff_files:
    print(f"  {file_path}")

print(f"\nAll files stored in: {order_dir}")
print(f"File paths available in 'local_tiff_files' array")

# The local_tiff_files array now contains Path objects for all downloaded tiff files
# You can use these for processing and delete them later when done

Processing order: scenes order, direct download
All files will be stored in: planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd
Order contains direct file links - downloading individual files...


planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd\PSScene\20250531_185113_13_24b9_3B_udm2.tif: 100%|██████████| 3.06k/3.06k [00:00<00:00, 8.62MB/s]


planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd\PSScene\20250531_185113_13_24b9_3B_AnalyticMS_metadata.xml: 100%|██████████| 0.01k/0.01k [00:00<00:00, 2.42MB/s]


planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd\PSScene\20250531_185113_13_24b9_3B_AnalyticMS.tif: 100%|██████████| 372k/372k [00:19<00:00, 20.4MB/s] 


planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd\PSScene\20250531_185113_13_24b9_metadata.json: 100%|██████████| 980/980 [00:00<00:00, 491kB/s]


...

planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd\PSScene\20250531_185109_51_24b9_3B_AnalyticMS_metadata.xml: 100%|██████████| 0.01k/0.01k [00:00<00:00, 1.91MB/s]


planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd\PSScene\20250531_185109_51_24b9_3B_AnalyticMS.tif: 100%|██████████| 366k/366k [00:21<00:00, 17.6MB/s] 


planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd\PSScene\20250531_185109_51_24b9_metadata.json: 100%|██████████| 977/977 [00:00<00:00, 347kB/s]


planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd\manifest.json: 100%|██████████| 0.01k/0.01k [00:00<00:00, 1.69MB/s]


Download complete! Found 6 SR tiff files:
  planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd\PSScene\20250531_185109_51_24b9_3B_AnalyticMS.tif
  planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd\PSScene\20250531_185111_32_24b9_3B_AnalyticMS.tif
  planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd\PSScene\20250531_185113_13_24b9_3B_AnalyticMS.tif
  planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd\PSScene\20250531_191447_74_2507_3B_AnalyticMS.tif
  planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd\PSScene\20250531_191449_94_2507_3B_AnalyticMS.tif
  planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd\PSScene\20250531_191452_15_2507_3B_AnalyticMS.tif

All files stored in: planet_downloads\6bb2fac9-ce05-4700-b5db-2504f28223dd
File paths available in 'local_tiff_files' array


## Publish to ArcGIS Online

Now the imagery can be published to ArcGIS Online!  Simply authenticate to ArcGIS Online, create a unique name for your imagery layer, and publish the imagery layer.

In [ ]:
# Connect to ArcGIS Online
gis = arcgis.gis.GIS(url="https://www.arcgis.com", client_id="8q9yiqWxN3WydTob")
# gis = arcgis.gis.GIS(url="https://www.arcgis.com", client_id="your-arcgis-client-id")
gis

In [ ]:
if not local_tiff_files:
    print("No tiff files found. Make sure to run the Planet download code first.")
else:
    # Create a unique timestamp
    timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    
    # Use timestamp and order name to create a unique name for the image collection
    # Extract a meaningful identifier from the first file name
    first_file_stem = local_tiff_files[0].stem
    scene_id = first_file_stem.split('_')[0]  # Get the scene ID part
    layer_name = f"PlanetLabs_{scene_id}_{timestamp}"
    
    print(f"Creating image collection: {layer_name}")
    print(f"Processing {len(local_tiff_files)} Planet images...")
    
    # Convert Path objects to strings for the API
    # Use the order directory directly since everything is now flat
    order_dir = download_dir / my_order_id
    input_folder = str(order_dir)  # Use the order-specific folder containing all images
    
    # Create the image collection using Planet imagery
    # Note: This can take significant time with large datasets
    try:
        published_imagery_layer = create_image_collection(
            image_collection=layer_name,
            input_rasters=input_folder,
            raster_type_name="Raster Dataset",  # Generic raster type - ArcGIS will detect metadata
            context={
                "outSR": {"wkid": 3857},  # Web Mercator projection
                "defineNodata": True,
                "noDataArguments": {
                    "noDataValues": [0],  # Planet uses 0 for nodata
                    "compositeValue": True
                },
                "buildFootprints": True,  # Enable footprints for better mosaicking
                "buildOverview": True,
                "image_collection_properties": {
                    "imageCollectionType": "Satellite"
                }
            },
            gis=gis
        )
        
        print(f"✅ Successfully created image collection: {layer_name}")
        print(f"📍 Image service URL: {published_imagery_layer.url}")
        print(f"🆔 Item ID: {published_imagery_layer.itemid}")
        
        # Store the result for later use
        planet_image_collection = published_imagery_layer
        
    except Exception as e:
        print(f"❌ Error creating image collection: {str(e)}")
        print("This might be due to:")
        print("- Insufficient ArcGIS Online privileges")
        print("- Raster analysis not configured properly")
        print("- Large dataset size (try with fewer images first)")

## View the New Image Service and Clean Up Folders
Now we can view the Image Service by drawing it on a map directly in this notebook!  Or you can view it in your ArcGIS Online environment.

#### Want to see the data now? [Check out this map here.](https://planetlabs.maps.arcgis.com/apps/instant/basic/index.html?appid=eaeaa5cf78694c63b9955ff7c87e479f)

In [ ]:
# view the new imagery layer on an arcgis map
# this layer can now be added to other maps, analyzed with ArcGIS Raster Analytics tools, and more

my_map = gis.map(location = planet_image_collection.extent, zoomlevel = 11)
my_map.basemap = "imagery"
my_map.add_layer(planet_image_collection)
my_map